# Problem statement

we are trying classify the data into one of the six classes(1-6)

### import packages

In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA # used for Dimensionality reduction
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_recall_fscore_support

In C:\Users\admin\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\admin\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\admin\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\admin\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\admin\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

### Data preparation

In [2]:
spend_data_input_file = "SpendData.csv"
test_data_input_file = "TestData.csv"
output_data_file = "predictions.csv"

In [3]:
spend_data = pd.read_csv(spend_data_input_file)
spend_data.head()

,Unnamed: 0,month,var8,var6,a.1,a.2,a.3,a.4,var5,b.5,...,c.276,c.277,c.278,c.279,c.280,c.281,c.282,c.283,f.284,t.158
0,1,1,2.0,NaN,1,5,1,57,34,1,...,1,0,0,0,1,0,0,0,5.0,NaN
1,2,1,2.0,NaN,1,4,1,57,34,2,...,0,0,0,0,0,0,1,0,NaN,NaN
2,3,1,2.0,NaN,1,5,1,57,42,2,...,0,0,0,0,0,0,0,0,3.0,NaN
3,4,1,2.0,NaN,1,6,1,57,34,2,...,0,0,0,0,1,0,1,0,5.0,NaN
4,5,1,2.0,NaN,1,8,1,22,1,1,...,0,0,0,0,0,0,0,0,5.0,NaN


In [4]:
# To view data types of all columns
spend_data.dtypes

Unnamed: 0      int64
month           int64
var8          float64
var6          float64
a.1             int64
               ...   
c.281           int64
c.282           int64
c.283           int64
f.284         float64
t.158         float64
Length: 301, dtype: object

In [5]:
# checking the number of rows and columns
spend_data.shape

(18379, 301)

In [6]:
# identifying columns which have more than 50% null values
missing_spend_data=spend_data.isnull().sum()
missing_spend_data[missing_spend_data > 9000]

var6     16480
b.10     16007
b.11     16007
b.13     11683
b.14     11920
b.20     14234
b.21     15098
c.31     15225
c.36     18148
c.37     18372
c.61     16770
c.67     17436
f.105    16159
f.106    16160
f.107    16378
f.108    17394
f.109    16542
f.110    17709
f.111    17450
f.112    15825
f.113    14983
f.114    17435
f.115    17878
f.116    17745
f.117    17985
f.118    17944
f.119    17821
f.120    18042
f.121    18128
f.122    17952
f.123    17874
a.184    18148
a.185    18372
b.195    14605
c.215    18148
c.216    18372
c.217    18264
c.218    18088
c.219    18297
c.220    18374
c.221    18350
c.222    15012
c.223    18292
t.158    18379
dtype: int64

### Missing Values Droping 

In [7]:
#Drop columns where percentage of missing values is greater than 50%
spend_data_wo_missing= spend_data.columns[spend_data.isnull().mean()>0.5]
spend_data.drop(spend_data_wo_missing, axis=1, inplace=True)

In [8]:
# here NaN will be dropped as they might effect data intregrity
spend_data.dropna(axis=1,inplace=True)
spend_data.isnull().sum()
spend_data.head()

,Unnamed: 0,month,a.1,a.2,a.3,a.4,var5,b.5,b.7,b.9,...,c.274,c.275,c.276,c.277,c.278,c.279,c.280,c.281,c.282,c.283
0,1,1,1,5,1,57,34,1,1,2,...,0,0,1,0,0,0,1,0,0,0
1,2,1,1,4,1,57,34,2,1,2,...,0,0,0,0,0,0,0,0,1,0
2,3,1,1,5,1,57,42,2,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,1,1,6,1,57,34,2,1,2,...,0,0,0,0,0,0,1,0,1,0
4,5,1,1,8,1,22,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### Data Preprocessing

In [9]:
# StandardScaler will transform your data such that its distribution will
# have a mean value close to 0 and standard deviation of 1
# after that we drop "pov6" because it is the target variable.

In [10]:
scaler=StandardScaler()
scaler.fit(spend_data.drop("pov6",axis=1))
scaler_data=scaler.transform(spend_data.drop("pov6",axis=1))
cols_list = list(spend_data.columns)
cols_list.remove("pov6")


In [11]:
scaled_spend_data=pd.DataFrame(scaler_data,columns=cols_list)
scaled_spend_data.head()

,Unnamed: 0,month,a.1,a.2,a.3,a.4,var5,b.5,b.7,b.9,...,c.274,c.275,c.276,c.277,c.278,c.279,c.280,c.281,c.282,c.283
0,-1.730735,-1.586804,0.0,-0.546694,0.0,0.008724,0.826032,-1.129119,-0.313106,0.384948,...,-0.368372,-0.161113,1.422973,-0.330312,-0.223849,-0.276121,2.454712,-0.199723,-0.275093,-0.13417
1,-1.730585,-1.586804,0.0,-0.954771,0.0,0.008724,0.826032,0.885646,-0.313106,0.384948,...,-0.368372,-0.161113,-0.702754,-0.330312,-0.223849,-0.276121,-0.407380,-0.199723,3.635138,-0.13417
2,-1.730434,-1.586804,0.0,-0.546694,0.0,0.008724,1.578506,0.885646,-0.313106,-2.597752,...,-0.368372,-0.161113,-0.702754,-0.330312,-0.223849,-0.276121,-0.407380,-0.199723,-0.275093,-0.13417
3,-1.730284,-1.586804,0.0,-0.138616,0.0,0.008724,0.826032,0.885646,-0.313106,0.384948,...,-0.368372,-0.161113,-0.702754,-0.330312,-0.223849,-0.276121,2.454712,-0.199723,3.635138,-0.13417
4,-1.730133,-1.586804,0.0,0.677539,0.0,-0.961049,-2.277924,-1.129119,-0.313106,-2.597752,...,-0.368372,-0.161113,-0.702754,-0.330312,-0.223849,-0.276121,-0.407380,-0.199723,-0.275093,-0.13417


# Dimensionality reduction

Dimensionality reduction, or dimension reduction, is the transformation of data from a high-dimensional space into a low-dimensional space so that the low-dimensional representation retains some meaningful properties of the original data, ideally close to its intrinsic dimension

###  Principal component analysis (PCA)

The main linear technique for dimensionality reduction, principal component analysis, performs a linear mapping of the data to a lower-dimensional space in such a way that the variance of the data in the low-dimensional representation is maximized

In [12]:
# Reducing total columns into 30 by using dimentionality reduction
pca=PCA(n_components=30)
pca.fit(scaled_spend_data)
pca_reduced_data=pca.transform(scaled_spend_data)
pca_reduced_data

array([[ 5.57698363, -3.89335611, -0.94552985, ..., -0.53061289,
        -0.36180427, -0.01798424],
       [ 1.79910696,  6.41254098, -3.77180523, ..., -0.14598645,
        -0.92055694,  1.02662837],
       [-1.82494567,  3.57095477,  6.01163025, ..., -0.44985455,
         2.98574962,  0.49421642],
       ...,
       [ 2.19782465,  2.45527838,  0.22010227, ...,  0.1576815 ,
        -0.29621345,  0.59453007],
       [ 2.16582051,  3.15431573,  0.38174457, ...,  0.83534377,
        -0.09625617,  0.18718402],
       [-1.41441381,  0.51256657,  0.20681426, ...,  1.05246761,
         0.38408659,  0.88204092]])

In [13]:
# converting resultant numpy array into usable data frame
reduced_df = pd.DataFrame(pca_reduced_data)
reduced_df = reduced_df.add_prefix("col_")
reduced_df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29
0,5.576984,-3.893356,-0.945530,-0.919697,4.903912,-2.261602,-1.695681,-1.108494,1.194786,-0.536978,...,1.000384,-1.042830,0.314454,1.916865,0.533286,-1.845991,2.030789,-0.530613,-0.361804,-0.017984
1,1.799107,6.412541,-3.771805,15.969092,16.860737,0.502792,-5.740049,-4.632112,0.054389,-4.023920,...,2.890813,-1.105261,0.319914,-2.517387,-0.795490,-0.025446,-1.066255,-0.145986,-0.920557,1.026628
2,-1.824946,3.570955,6.011630,5.893016,0.941952,-2.000453,2.175062,-1.006403,0.745248,-2.531259,...,0.906858,-1.377063,-1.319699,0.337589,4.492859,-1.338828,-1.161413,-0.449855,2.985750,0.494216
3,3.378573,5.669926,-0.619704,3.190359,2.223196,-0.236702,-0.580689,-0.532331,-0.770146,0.706146,...,-0.863749,1.555289,0.389264,-0.179840,0.269530,2.055052,-1.111648,0.288829,-0.830762,1.765772
4,-0.682802,1.448911,1.189268,0.652828,-0.876857,-2.258717,2.277323,-0.813848,1.122941,-1.218312,...,0.331821,0.413713,-1.562875,0.773678,3.502273,-1.649697,-0.542635,0.591468,0.893808,1.386373


In [14]:

X = reduced_df
y = spend_data["pov6"]

In [15]:
# Splitting the dataset into the Training set and Test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

###  LinearSVC
Implementation of Support Vector Machine classifier using libsvm: the kernel is linear and it can scale large number of samples.

In [16]:
classifier=LinearSVC()
classifier.fit(x_train,y_train)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [17]:
y_pred=classifier.predict(x_test)

In [18]:
#calculating accuracy using sklearn.metrics.accuracy_score
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9208378672470077

the accurasy is 92.2% but we look at precission and Recall and Fscore to  validate the results 
and  avoid model overfitting 

In [19]:
precision_recall_fscore_support(y_test,y_pred, average="weighted")
# we  are using  "weighted"  as the average parameter value to account for label imbalance;

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.9124244240087198, 0.9208378672470077, 0.9081012710634417, None)

####  Precision

Precision is the ratio of system generated results that correctly predicted positive observations (True Positives) to the system’s total predicted positive observations, both correct (True Positives) and incorrect (False Positives). 


In [20]:
# precision = 0.9134728928549671

#### Recall (aka Sensitivity)

Recall is the ratio of system generated results that correctly predicted positive observations (True Positives) to all observations in the actual malignant class (Actual Positives).

In [21]:
# Recall = 0.9224700761697497

#### F1 Score

The F1 Score is the weighted average (or harmonic mean) of Precision and Recall. Therefore, this score takes both False Positives and False Negatives into account to strike a balance between precision and Recall.  

In [22]:
#F1 score = 0.9110047201638984

# Test data

# Loading the Test Data

In [23]:
test_data= pd.read_csv(test_data_input_file)
test_data.head()

,Unnamed: 0,month,var8,var6,a.1,a.2,a.3,a.4,var5,b.5,...,c.275,c.276,c.277,c.278,c.279,c.280,c.281,c.282,c.283,f.284
0,9,1,1.0,NaN,1,11,1,47,8,2,...,0,0,0,0,0,0,0,1,0,3.0
1,15,1,2.0,NaN,1,11,1,65,27,1,...,0,1,0,0,0,0,0,0,0,3.0
2,16,1,2.0,NaN,1,6,1,65,27,2,...,0,0,0,0,0,0,0,1,0,3.0
3,24,1,1.0,NaN,1,3,1,3,2,1,...,0,0,0,0,0,0,0,0,0,NaN
4,32,1,1.0,NaN,1,5,1,83,3,1,...,0,1,0,0,0,1,0,0,0,NaN


In [24]:
# To view the data types
test_data.dtypes

Unnamed: 0      int64
month           int64
var8          float64
var6          float64
a.1             int64
               ...   
c.280           int64
c.281           int64
c.282           int64
c.283           int64
f.284         float64
Length: 300, dtype: object

In [25]:
# retaining columns from train data in test before dimentionality reduction
# "totshopping.rep" column does not exist in test data
cols_list.remove("totshopping.rep")
test_data=test_data[[column for column in cols_list]]

In [26]:
test_data.shape

(4595, 226)

In [27]:
# scaling the data to values between 0 and 1
scaler=StandardScaler()
scaler.fit(test_data)
scaler_test=scaler.transform(test_data)

In [28]:
#dimentionality reduction
pca=PCA(n_components=30)
pca.fit(scaler_test)
pca_reduced_data=pca.transform(scaler_test)

In [29]:
scaler_test = pd.DataFrame(pca_reduced_data)
scaler_test = scaler_test.add_prefix("col_")
scaler_test.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,...,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29
0,-1.417485,-0.974079,-4.002625,2.378035,5.511175,-2.552030,2.400070,-1.226055,0.447775,-1.057105,...,0.877445,-0.464952,-0.284146,1.319424,-0.037041,-0.854157,0.506371,1.009159,-0.711353,0.164075
1,1.603641,4.036920,-0.415667,-1.086715,2.245215,-2.701981,1.203774,-2.456401,-0.859836,-0.241992,...,0.133331,0.217151,0.104035,-0.546212,2.061417,0.533317,-1.138466,-1.783662,-2.858297,0.909654
2,0.846427,-9.421028,-0.977285,11.924552,20.248744,-10.720655,2.997815,-0.608225,1.315620,-0.475269,...,-1.875166,2.969475,-1.301935,-4.084559,-2.123635,2.922390,1.270396,-2.106607,-1.047432,-0.009647
3,-1.764442,1.138955,-2.891490,-1.220625,1.078658,-1.590576,2.164890,-1.445554,0.980106,-0.510529,...,0.228030,0.075364,-0.366981,-0.252304,-0.096460,0.442403,0.087941,0.004648,0.533727,0.184158
4,0.404474,-1.057682,-3.065356,-2.940054,0.530382,-1.503879,2.733837,-1.772230,0.771440,-0.442375,...,0.850641,-0.198590,-0.216495,-0.061990,-0.170975,0.509539,0.587380,-0.046025,-0.832731,0.365549


In [30]:
# predicting the values for test data
y_pred=classifier.predict(scaler_test)

In [31]:
# creating the CSV  with the prediction
pd.DataFrame(y_pred).to_csv(output_data_file)

# Explonations

In [32]:
a.Please explain the choice of metric / evaluation criterion used

   Accuracy classification score for validation data against model predictions
   evaluation criterion used are precession, Recall, Fscore.

b.What are the assumptions you made when building this model?
  
    drop null values assuming they would conflict model performance

c.What were the approaches you considered?Please explain the reason for the technique/ approach used as well as the pros and cons.
    
    the first approach was dropping null values 
     pros: model performances is not effected
     cons: loss of data 
    the second approach was dimentionality reduction
     pros: reducess the number of columns
     cons: original data loss
    the 3rd approach is linear SVC
     pros: faster than other models
     cons: it does not take categorical variables
        
d. Please explain under what conditions will the model you choose be not appropriate
    if the number of records is greater than 100k then the model is not appropriate.


e. How confident are you of the model’s robustness and how would you explainthemodel’s performance?
    if new labels data is pass then the model will predict its label to be one of the 6labels  
        
        
        
f. Why is your model performing well / not well?       
    the model shows the accurasy score of 92.2%, along with that precession Recall and fscore 
    builds confidences in the model. the model performing well because there are less than 
    100k records, all the values have been scaled  and we did wieghted calculations
    of evaluation metrics.
        
        
g. Was any feature engineering required? If yes, what were they. If no, why?        
    1).Removing features with more than 50% null values.
    2).dimentionality reduction to reduce the number of columns from actual numbers to 30.
       30 after testing with other values in the range 25 to 40.
    
    

SyntaxError: invalid syntax (<ipython-input-32-07ecff6193dc>, line 1)